In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import warnings
import spacy
from anchor import anchor_text
import pickle
from myUtils import *
from transformer.utils import *
from datasets.dataset_loader import *

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

In [3]:
plt.rcParams['font.size'] = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [68]:
review_parser, label_parser, ds_train, ds_valid, ds_test = create_sentiment_dataset()

Number of tokens in training samples: 3307
Number of tokens in training labels: 2


In [69]:
anchor_examples = pickle.load( open( "results/transformer_anchor_examples.pickle", "rb" ))
explanations = pickle.load(open( "results/gru_extended_counter_exps.pickle", "rb" ))
explanations = [exp for exp in explanations if len(exp.fit_examples) > 10] 
explanations.sort(key=lambda exp: exp.test_precision)

In [70]:
def generate_example(ds, anchor_examples, explanations):
    filtered_examples = []
    for example in ds:
        for exp in explanations:
            if ' '.join(example.text)==anchor_examples[exp.index]:
                if exp.test_precision < 0.5:
                    example.label = 0
                else:
                     example.label = 2*(exp.test_precision - 0.5)
                filtered_examples.append(example)
                
                break
                
    return filtered_examples

In [71]:
examples = generate_example(ds_train, anchor_examples, explanations)

In [75]:
len(explanations)

228

In [41]:
size = len(examples)
train_size = round(0.75*size)
train_examples = examples[:train_size]
valid_examples = examples[train_size:]

In [43]:
ds_train.examples = train_examples
ds_valid.examples = valid_examples
ds_test = ds_valid